<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation

## *Data Science Unit 4 Sprint 2 Lesson 2*


### A Review of Yesterday's Content:

- Neural Networks are made up of layers of nodes or neurons. 
- There are three kinds of layers: input, hidden, and output layers.
<center><img src="https://cdn-images-1.medium.com/max/1600/1*_M4bZyuwaGby6KMiYVYXvg.jpeg" width="400"></center>
- Each layer is made up of individual neurons which have a corresponding weight and bias.
<center><img src="https://i.stack.imgur.com/6S6Bz.png" width="400"></center>
- Signal is passed from layer to layer through a network by:
 - Taking in inputs from the training data (or previous layer)
 - Multiplying each input by its corresponding weight (think arrow/connecting line)
 - Adding a bias to this weighted some of inputs and weights
 - Activating this weighted sum + bias by squishifying it with sigmoid or some other activation function. With a single perceptron with three inputs, calculating the output from the node is done like so:
\begin{align}
 y = sigmoid(\sum(weight_{1}input_{1} + weight_{2}input_{2} + weight_{3}input_{3}) + bias)
\end{align}
 - this final activated value is the signal that gets passed onto the next layer of the network.
 
 
## Learning Objectives
* <a href="#p1">Part 1</a>: Explain the intutition behind backproprogation
* <a href="#p2">Part 2</a>: Implement backpropagation on a feedforward neural network. 


## Multilayer Perceptrons AKA Feedforward Neural Networks

Yesterday we talked about the capabilities and limitations of individual perceptrons.

- Perceptrons are like a single neuron in a neural network
- Perceptrons can only fit linear boundaries between classes

What is a Feedforward Neural Network? Well, it's a Neural Network made up of multiple perceptrons that has at least 1 hidden layer (does not include input and output layers). When we use the [Tensorflow Playground](https://playground.tensorflow.org) website to explore different neural network architectures, we're really just composing different architectures (topologies) of feedforward neural networks. However, when we give the network multiple hidden layers it should be designated as a "deep" feedforward neural network.

### Perceptron

<center><img src="http://www.ryanleeallred.com/wp-content/uploads/2019/04/Perceptron.png" width="300"></center>

### Feedforward Neural Network

<center><img src="http://www.ryanleeallred.com/wp-content/uploads/2019/04/Feedforward-NN.png" width="300"></center>

### Deep Feedforward Neural Network
<center><img src="http://www.ryanleeallred.com/wp-content/uploads/2019/04/Deep-Feedforward-NN.png" width="400"></center>


# How do Neural Networks Learn?

## Calculating *"loss"* or *"error"*

We've talked about how in order to evaluate a network's performance, the data is "fed forward" until predictions are obtained and then the "loss" or "error" for a given observation is ascertained by looking at what the network predicted for that observation and comparing it to what it *should* have predicted. 

The error for a given observation is calculated by taking the square of the difference between the predicted value and the actual value. 

We can summarize the overal quality of a network's predictions by finding the average error across all observations. This gives us the "Mean Squared Error." which hopefully is a fairly familiar model evaluation metric by now. Graphing the MSE over each epoch (training cycle) is a common practice with Neural Networks. This is what you're seeing in the top right corner of the Tensorflow Playground website as the number of "epochs" climbs higher and higher.

We also graphed loss over a number of iterations when working with gradient descent. Neural Networks use gradient descent to find their optimal parameters.

<center><img src="http://cs231n.github.io/assets/nn3/learningrates.jpeg" width="400"></center>

## What is an "Epoch"?

An "Epoch" is one cycle of passing our data forward through the network, measuring error given our specified cost function, and then -via gradient descent- updating weights within our network to hopefully improve the quality of our predictions on the next iteration.

### Batch vs Minibatch vs Stochastic Gradient Descent Epochs

You may have heard these variations on the training process referenced in the 3Blue1Brown videos about backpropagation. "Minibatch" Gradient Descent means that instead of passing all of our data through the network for a given epoch (Batch GD), we just pass a randomized portion of our data through the network for each epoch. 

Stochastic Gradient Descent is when we make updates to our weights after forward propagating each individual training observation.

### Why might we use something like Minibatch GD? 

Neural Networks often require high quantities of training data. Consider the [MNIST Dataset](http://yann.lecun.com/exdb/mnist/):

![MNIST](https://corochann.com/wp-content/uploads/2017/02/mnist_plot-800x600.png)

This dataset is what has been used in the 3Blue1Brown videos for demonstrating important neural network concepts. The dataset has 60,000 training observations and 10,000 test observations. SInce there are 10 numeric digits that works out to be 6,000 training images per digit. Do you think our neural network needs to look at all 6000 number 3s in order to get an idea of how accurate or innacurate it is at recognizing the number 3? No, it doesn't. It's actually pretty computationally expensive to run the entire dataset through the network for each epoch, so we'll just randomly select a portion of the dataset for each training epoch.

How much of the data should you pass through for each epoch? Well, you'll probably get sick of hearing this response to questions like this, but `batch_size` is another hyperparameter for our model that we'll need to tune to try and get the best results. We'll try different amounts out and try to find an optimal "batch size."

### A note about Hyperparameters

Neural Networks have many more hyperparameters than other machine learning algorithms which is part of what makes them a beast to train. 1) You need more data to train them on. 2) They're complex so they take longer to train. 3) They have lots and lots of hyperparameters which we need to find the most optimal combination of, so we might end up training our model dozens or hundreds of times with different combinations of hyperparameters in order to try and squeeze out a few more tenths of a percent of accuracy. 

# Backpropagation

Backpropagation is short for ["Backwards Propagation of errors"](https://en.wikipedia.org/wiki/Backpropagation) and refers to a specific (rather calculus intensive) algorithm for how weights in a neural network are updated in reverse order at the end of each training epoch. Our purpose today is to demonstrate the backpropagation algorithm on a simple Feedforward Neural Network and in so doing help you get a grasp on the main process. If you want to understand all of the underlying calculus of how the gradients are calculated then you'll need to dive into it yourself, [3Blue1Brown's video is a great starting place](https://www.youtube.com/watch?v=tIeHLnjs5U8). I also highly recommend this Welch Labs series [Neural Networks Demystified](https://www.youtube.com/watch?v=bxe2T-V8XRs) if you want a rapid yet orderly walkthrough of the main intuitions and math behind the backpropagation algorithm. 



## FeedForward NN for predicting Test Scores

![231 Neural Network](https://cdn-images-1.medium.com/max/1600/1*IjY3wFF24sK9UhiOlf36Bw.png)

### Generate some Fake Data

In [ ]:
import numpy as np

np.random.seed(812)

# hours studying, hours sleep
X = np.array(([2,9], 
              [1,5],
              [3,6]), dtype=float)

# Exam Scores
y = np.array(([92],
              [86],
              [89]), dtype=float)

In [ ]:
# Normalizing Data on feature 
# Neural Network would probably do this on its own, but it will help us converge on a solution faster

X = X / np.amax(X, axis=0)

y = y / 100

print("Studying, Sleeping \n", X)
print("Test Score \n", y)

### Lets create a Neural_Network class to contain this functionality

In [ ]:
class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.input = 2
        self.hiddenNodes = 3
        self.outputNodes = 1
        
        # Initial Weights
        # 2x3 Matrix Array for the First Layer
        self.weights1 = np.random.randn(self.input,self.hiddenNodes)
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)

### How many random weights do we need to initialize? "Fully-connected Layers"

In [4]:
nn = NeuralNetwork()

In [5]:
print("Layer 1 wieghts: \n", nn.weights1)
print("Layer 2 wieghts: \n", nn.weights2)

Layer 1 wieghts: 
 [[ 2.48783189  0.11697987 -1.97118428]
 [-0.48325593 -1.50361209  0.57515126]]
Layer 2 wieghts: 
 [[-0.20672583]
 [ 0.41271104]
 [-0.57757999]]


### Implement Feedforward Functionality

After this step our neural network should be able to generate an output even though it has not been trained.

In [6]:
class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.input = 2
        self.hiddenNodes = 3
        self.outputNodes = 1
        
        # Initial Weights
        # 2x3 Matrix Array for the First Layer
        self.weights1 = np.random.randn(self.input,self.hiddenNodes)
        # 3x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def feed_forward(self,X):
        """
        Calculate the NN inference using feed forward.
        """
        
        # Weighted sum of inputs & hidden
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final Activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output

### Can we generate an output?

In [7]:
nn = NeuralNetwork()

print(X[0])
output = nn.feed_forward(X[0])
print("output", output)

[0.66666667 1.        ]
output [0.25814933]


### Calculating Error

In [8]:
error = y[0] - output
error

array([0.66185067])

In [9]:
output_all = nn.feed_forward(X)
error_all = y - output_all
print(output_all)
print(error_all)

[[0.25814933]
 [0.33067192]
 [0.22642076]]
[[0.66185067]
 [0.52932808]
 [0.66357924]]


## Backpropagation (Assigning blame for bad predictions and delivering justice - repeatedly and a little bit at a time)

What in our model could be causing our predictions to suck so bad? 

Well, we know that our inputs (X) and outputs (y) are correct, if they weren't then we would have bigger problems than understanding backpropagation.

We also know that our activation function (sigmoid) is working correctly. It can't be blamed because it just does whatever we tell it to and transforms the data in a known way.

So what are the potential culprits for these terrible predictions? The **weights** of our model. Here's the problem though. I have weights that exist in both layers of my model. How do I know if the weights in the first layer are to blame, or the second layer, or both? 

Lets investigate. And see if we can just eyeball what should be updated.

In [10]:
attributes = ['weights1', 'hidden_sum', 'activated_hidden', 'weights2', 'output']

[print(i + '\n', getattr(nn,i), '\n'+'---'*3) for i in dir(nn) if i in attributes]

activated_hidden
 [[0.22536165 0.75468678 0.7231884 ]
 [0.34955543 0.64075804 0.6281894 ]
 [0.14218011 0.8075341  0.68484697]] 
---------
hidden_sum
 [[-1.23468981  1.12376697  0.96033266]
 [-0.62099392  0.57865576  0.52445712]
 [-1.79729952  1.4340663   0.77613709]] 
---------
weights1
 [[-1.75351135  1.23279898  0.24464757]
 [-0.06568225  0.30190098  0.79723428]] 
---------
weights2
 [[ 1.23073545]
 [-1.52187331]
 [-0.25502715]] 
---------


[None, None, None, None]

In [11]:
# We want activated that correspond to negative weights to be lower
# And we want more higher activation for positivie weights

Why is my error so big?

My error is so big because my prediction is low.

Why are my prediction low?

Because either:

  1) Second layer **weights** are low
  
  (or)
  
  2) Activations coming from the first layer are low
  
How are activations from the first layer determined? 

  1) By inputs - fixed
  
  2) by **weights** - variable
  
The only thing that I have control over throughout this process in order to increase the value of my final predictions is to either increase weights in layer 2 or increase weights in layer 1. 

Imagine that you could only change your weights by a fixed amount. Say you have .3 and you have to split that up and disperse it over your weights so as to increase your predictions as much as possible. (This isn't actually what happens, but it will help us identify which weights we would benefit the most from moving.)

I need to increase weights of my model somewhere, I'll get the biggest bang for my buck if I increase weights in places where I'm already seeing high activation values -because they end up getting multiplied together before being passed to the sigmoid function. 

> "Neurons that fire together, wire together"

In [12]:
# I want activations that correspond to negative weights to be lower
# and activations that correspond to positive weights to be higher

class NeuralNetwork: 
    def __init__(self):
        # Set upArchietecture 
        self.inputs = 2
        self.hiddenNodes = 3
        self.outputNodes = 1
        
        #Initial weights
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes) #2x3
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes) #3x1
    
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        #Weighted sume of inputs and hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        #Acivations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        #Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        self.o_error = y - o #error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid to error
        
        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [13]:
# WHAT WE DID ABOVE IN THE FUNCTION JUST BROKEN DOWN


In [14]:
# LETS LOOK AT THE SHAPE OF HTE GRADIENT COMPONETS

nn = NeuralNetwork()
nn.train(X,y)

In [15]:
# NOW LETS LOOK AT OUR ERROR THAT IS ASSOCIATED WITH EACH OBSERVATION
# aka how wrong were we??

In [16]:
nn.o_error

array([[0.26708134],
       [0.1933596 ],
       [0.25018053]])

In [17]:
# 1st Gradient

# SIMPLE INTERPRETATION--  NOW WE WANT TO KNOW HOW MUCH MORE SIGMOID ACTIVATION WOULD WE NEED TO PUSH US TOWARDS THE CORRECT ANSWER?

nn.o_delta

array([[0.06052487],
       [0.04297049],
       [0.05765423]])

In [18]:
# 2nd ERROR
# THE FINGER POINTING ISNT DONE YET  WE STILL HAVE NEURONS TO SNITCH ON. LETS GO BACK ANOTHER LAYER 
# THE SHAPE HERE IS GOING TO CORRESPOND TO THE NUMBER OF OBSERVATIONS AND THE NUMBER OF WEIGHTS PER THE HIDDEN OUTPUT LAYER

nn.z2_error

array([[0.02513232, 0.02388093, 0.04236108],
       [0.01784304, 0.01695461, 0.03007485],
       [0.02394031, 0.02274828, 0.04035193]])

In [19]:
# 2nd Gradient
nn.z2_delta

array([[0.00577351, 0.00552659, 0.01058153],
       [0.00431831, 0.00417582, 0.00751868],
       [0.00595272, 0.00369043, 0.00957855]])

In [20]:
# NOW WE HAVE CALCULATED THE GRADIENTS NOW ITS TIME TO CALCULATE THE DESCENT. MEANING THAT NOW WE ARE GOING TO LOOK AT THE DIRECTION 
# THAT WE HAVE CALCULATED FROM BACK PROPERGATION AND DESCEND OUR LOSS FUNCTION ACCORDING TO THOSE GRADIENTS. 

# I KNOW HAVE ENOUGH INFORMATION TO UPDATE THE WEIGHTS BECAUSE NOW I KNOW WHO IS RESPONSIBLE FOR THE AMOUNT OF ERROR THAT I HAVE 
# AND WHAT DIRECTION I CAN UPDATE MY WEIGHTS TO INCREASE THE PREFORMANCE 

# WE ARE GOING TO UPDATE OUR INPUTS TO OUR HIDDEN WEIGHTS FIRST
X.T.dot(nn.z2_delta)

array([[0.01124117, 0.00876677, 0.01913914],
       [0.01214105, 0.01030678, 0.02114428]])

In [21]:
nn = NeuralNetwork()

for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 1---------+
Input: 
 [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output: 
 [[0.92]
 [0.86]
 [0.89]]
Predicted Output: 
 [[0.77107435]
 [0.74476221]
 [0.77750324]]
Loss: 
 0.016038039613679767
+---------EPOCH 2---------+
Input: 
 [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output: 
 [[0.92]
 [0.86]
 [0.89]]
Predicted Output: 
 [[0.78753123]
 [0.76063943]
 [0.7944179 ]]
Loss: 
 0.012185478517421952
+---------EPOCH 3---------+
Input: 
 [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output: 
 [[0.92]
 [0.86]
 [0.89]]
Predicted Output: 
 [[0.80053966]
 [0.77329478]
 [0.8077293 ]]
Loss: 
 0.009519012095819157
+---------EPOCH 4---------+
Input: 
 [[0.66666667 1.        ]
 [0.33333333 0.55555556]
 [1.         0.66666667]]
Actual Output: 
 [[0.92]
 [0.86]
 [0.89]]
Predicted Output: 
 [[0.81108453]
 [0.7836281 ]
 [0.81847816]]
Loss: 
 0.007603539627961725
+---------EP

## Diagnosing Backpropagation issues by hand. (not recommended)

Our model has 9 total weights (6 in the first layer, 3 in the last layer) that could be off.

1) Calculate Error for a given each observation

2) Does the error indicate that I'm overestimating or underestimating in my prediction?

3) Look at final layer weights to get an idea for which weights are helping pass desireable signals and which are stifling desireable signals

4) Also go to the previous layer and see what can be done to boost activations that are associated with helpful weights, and limit activations that are associated with unhelpful weights.

### However, we would prefer to do all this in a fancy automated, controlled calculus way.

5) Repeat steps 1-4 for every observation in a given batch, and then given the network's cost function, calculate its gradient using calculus and update weights associated with the (negative) gradient of the cost function. 

Remember that we have 9 weights in our network therefore the gradient that comes from our gradient descent calculation will be the vector that takes us in the most downward direction along some function in 9-dimensional hyperspace.

\begin{align}
C(w1, w2, w3, w4, w5, w6, w7, w8, w9)
\end{align}

You should also know that with neural networks it is common to have gradients that are not convex (like what we saw when we applied gradient descent to linear regression). Due to the high complexity of these models and their nonlinearity, it is common for gradient descent to get stuck in a local minimum, but there are ways to combat this:

1) Stochastic Gradient Descent

2) More advanced Gradient-Descent-based "Optimizers" - See Stretch Goals on assignment.


## Adding BackPropogation to our implementation (non Gradient-Descent)

## A true GD-based implementation from [Welch Labs](https://www.youtube.com/watch?v=bxe2T-V8XRs)

In [22]:
class Neural_Network(object):
    def __init__(self):        
        #Define Hyperparameters
        self.inputLayerSize = 2
        self.outputLayerSize = 1
        self.hiddenLayerSize = 3
        
        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self, X):
        #Propogate inputs though network
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        yHat = self.sigmoid(self.z3) 
        return yHat
        
    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)
        return J
        
    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forward(X)
        
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(X.T, delta2)  
        
        return dJdW1, dJdW2
    
    #Helper Functions for interacting with other classes:
    def getParams(self):
        #Get W1 and W2 unrolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single paramater vector.
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, X, y):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [23]:
from scipy import optimize
class trainer(object):
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N
        
    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X, self.y))   
        
    def costFunctionWrapper(self, params, X, y):
        self.N.setParams(params)
        cost = self.N.costFunction(X, y)
        grad = self.N.computeGradients(X,y)
        
        return cost, grad
        
    def train(self, X, y):
        #Make an internal variable for the callback function:
        self.X = X
        self.y = y

        #Make empty list to store costs:
        self.J = []
        
        params0 = self.N.getParams()

        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS', \
                                 args=(X, y), options=options, callback=self.callbackF)

        self.N.setParams(_res.x)
        self.optimizationResults = _res

In [24]:
NN = Neural_Network()

In [25]:
T = trainer(NN)

In [26]:
T.train(X,y)

Optimization terminated successfully.
         Current function value: 0.000178
         Iterations: 57
         Function evaluations: 62
         Gradient evaluations: 62


In [27]:
print("Predicted Output: \n" + str(NN.forward(X))) 
print("Loss: \n" + str(np.mean(np.square(y - NN.forward(X))))) # mean sum squared loss

Predicted Output: 
[[0.90563279]
 [0.87216606]
 [0.89083303]]
Loss: 
0.00011837461852817363


In [28]:
import matplotlib.pyplot as plt
plt.plot(T.J)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()     

<Figure size 640x480 with 1 Axes>